In [1]:
## Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout 
from tensorflow.keras import metrics, regularizers
from tensorflow.keras.layers import BatchNormalization

In [3]:
## importing the data and preview
df_train = pd.read_csv('train.csv')
#df_test = pd.read_csv('test.csv')
df_y= pd.read_csv('train_labels.csv')
df_train.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [4]:
## Data description
df_train.describe()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
count,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06,1.558080e+06
mean,1.298350e+04,3.316331e+02,2.950000e+01,4.365526e-04,-1.034982e-03,-2.178045e-01,-2.156555e-03,-1.828903e-03,-1.651785e-03,-4.122917e-04,-2.620665e-05,-1.298393e-04,1.365584e-03,3.315801e-04,-3.733291e-03,-1.172605e-02
std,7.496318e+03,1.958257e+02,1.731811e+01,2.658684e+00,4.404200e+00,2.298002e+00,3.934184e+00,1.683685e+00,1.590818e+00,3.345143e+00,3.243428e+00,4.501534e+00,2.592913e+00,1.917333e+00,4.532568e+00,3.911767e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-3.750634e+02,-4.345977e+02,-3.165948e+01,-4.083761e+02,-2.362601e+01,-7.498280e+01,-4.705046e+02,-4.070115e+02,-5.361000e+02,-2.703468e+02,-4.341271e+01,-4.270586e+02,-6.125494e+02
25%,6.491750e+03,1.617500e+02,1.475000e+01,-5.000000e-01,-4.831933e-01,-6.461531e-01,-4.929204e-01,-4.729928e-01,-4.786836e-01,-4.927140e-01,-5.022901e-01,-5.000000e-01,-5.151734e-01,-4.787939e-01,-4.835391e-01,-5.805627e-01
50%,1.298350e+04,3.350000e+02,2.950000e+01,-3.091190e-03,3.151261e-03,0.000000e+00,0.000000e+00,-1.589577e-03,2.991773e-03,9.107468e-04,-2.290076e-03,0.000000e+00,-1.445087e-03,-1.655822e-03,3.086420e-03,0.000000e+00
75%,1.947525e+04,5.010000e+02,4.425000e+01,4.845440e-01,4.926471e-01,3.338469e-01,4.893805e-01,4.701565e-01,5.056096e-01,4.927140e-01,4.847328e-01,5.000000e-01,5.086705e-01,4.780386e-01,4.938272e-01,5.703325e-01
max,2.596700e+04,6.710000e+02,5.900000e+01,3.358246e+02,4.495914e+02,1.666667e+00,4.366504e+02,2.487286e+01,7.791548e+01,4.425009e+02,3.312542e+02,6.301000e+02,3.679812e+02,4.186559e+01,4.480206e+02,6.305111e+02


In [5]:
##  Checking for null values
df_train.isnull().sum()

sequence     0
subject      0
step         0
sensor_00    0
sensor_01    0
sensor_02    0
sensor_03    0
sensor_04    0
sensor_05    0
sensor_06    0
sensor_07    0
sensor_08    0
sensor_09    0
sensor_10    0
sensor_11    0
sensor_12    0
dtype: int64

In [6]:
df_y.shape

(25968, 2)

In [7]:
df_y.isnull().sum()

sequence    0
state       0
dtype: int64

In [8]:
df_y.tail()

,sequence,state
25963,25963,1
25964,25964,0
25965,25965,1
25966,25966,1
25967,25967,0


In [9]:
# train test split 20%
c = round(len(df_train.sequence.unique())*0.8)
df_Xtrain = df_train[df_train['sequence'] <= c]
df_Xtest = df_train[df_train['sequence'] > c]
df_ytrain = df_y[df_y.sequence <= (df_Xtrain.iloc[-1,0])]
df_ytest = df_y[df_y.sequence > (df_Xtrain.iloc[-1,0])]

In [10]:
# Scaling
scaler = StandardScaler()
df_Xtrain.iloc[:,3:] = scaler.fit_transform(df_Xtrain.iloc[:,3:])
df_Xtest.iloc[:,3:] = scaler.fit_transform(df_Xtest.iloc[:,3:])

In [11]:
## Preparing the data for making 3D arrays for LSTM

X_train=[]
for i in df_Xtrain.sequence.unique():
    X_train.append(df_Xtrain[df_Xtrain.sequence == i].iloc[:,3:])

X_test=[]
for i in df_Xtest.sequence.unique():
    X_test.append(df_Xtest[df_Xtest.sequence == i].iloc[:,3:])

target=[]
for i in df_test.sequence.unique():
    target.append(df_test[df_test.sequence == i].iloc[:,3:]) 


X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(df_ytrain.state)
Y_test = np.array(df_ytest.state)
target = np.array(target)

NameError: name 'df_test' is not defined

In [15]:
## Data shape
X_train.shape

(20775, 60, 13)

In [26]:
# model defining

model = Sequential()
model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=[X_train.shape[1],X_train.shape[2]]))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
#model.add(Dropout(0.2))
#model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation= 'sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_15 (Bidirect  (None, 60, 512)           552960    
 ional)                                                          
                                                                 
 bidirectional_16 (Bidirect  (None, 60, 256)           656384    
 ional)                                                          
                                                                 
 bidirectional_17 (Bidirect  (None, 128)               164352    
 ional)                                                          
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1373825 (5.24 MB)
Trainable params: 1373825 (5.24 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [28]:
## training the model
training_start = time.perf_counter()
model.fit(X_train,Y_train, epochs=15,batch_size=256, validation_split=0.1, verbose=1)
training_end = time.perf_counter()

Epoch 1/15
74/74 [==============================] - 141s 2s/step - loss: 0.3927 - accuracy: 0.8301 - val_loss: 0.4744 - val_accuracy: 0.7782
Epoch 2/15
74/74 [==============================] - 141s 2s/step - loss: 0.3630 - accuracy: 0.8426 - val_loss: 0.4143 - val_accuracy: 0.8205
Epoch 3/15
74/74 [==============================] - 142s 2s/step - loss: 0.3340 - accuracy: 0.8569 - val_loss: 0.4566 - val_accuracy: 0.7964
Epoch 4/15
74/74 [==============================] - 141s 2s/step - loss: 0.3192 - accuracy: 0.8655 - val_loss: 0.3653 - val_accuracy: 0.8335
Epoch 5/15
74/74 [==============================] - 142s 2s/step - loss: 0.3043 - accuracy: 0.8708 - val_loss: 0.3749 - val_accuracy: 0.8373
Epoch 6/15
74/74 [==============================] - 140s 2s/step - loss: 0.2789 - accuracy: 0.8846 - val_loss: 0.3548 - val_accuracy: 0.8508
Epoch 7/15
74/74 [==============================] - 143s 2s/step - loss: 0.2445 - accuracy: 0.9002 - val_loss: 0.3518 - val_accuracy: 0.8479
Epoch 8/15
74

In [29]:
## model evaluation

test_loss, test_accuracy = model.evaluate(X_test, Y_test)
test_pred = model.predict(X_test)
test_pred = np.round(test_pred)
precision = precision_score(test_pred, Y_test)
f_score = f1_score(test_pred, Y_test)
recall = recall_score(test_pred, Y_test)
accuracy = accuracy_score(test_pred, Y_test)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f_score)
print("Accuracy:", accuracy)

163/163 [==============================] - 27s 149ms/step
Precision: 0.8728189220628151
Recall: 0.8858717040535222
F1-score: 0.879296875
Accuracy: 0.8809936452917388
